In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from IPython.display import clear_output
import time
plt.rcParams.update({'font.size': 14})

from utilities import error, prepare_phi_and_S, convert1D_to_2D, plot_phi,residual

In [ ]:
def GS(Nx,Ny,phi,S,aE,aW,aN,aS,a0):
    for i in range(1,Nx-1):
            for j in range(1,Ny-1):

                # Gauss-Siedel Update
                phi[i,j] = (S[i,j] - aE*phi[i+1,j] - aW*phi[i-1,j] - aN*phi[i,j+1] - aS*phi[i,j-1]) / a0
    return phi

In [ ]:
# numbering scheme used is k = (j-1)*N + i
# The name of variable ends with c and f are corse and fine mesh
start = time.time()
Nxf = 41
Nyf = 41
Nxc = 21
Nyc = 21
L = 1   # length
H = 1   # length

Sf = np.zeros((Nxf,Nyf))
phif = np.zeros((Nxf,Nyf))

Sc = np.zeros((Nxc,Nyc))
phic = np.zeros((Nxc,Nyc))

dxf = L/(Nxf-1)    # Grid size
dyf = L/(Nyf-1)    # Grid size

dxc = L/(Nxf-1)    # Grid size
dyc = L/(Nyf-1)    # Grid size

tolc = 1e-6

aEf = 1/dxf**2
aWf = 1/dxf**2
aNf = 1/dyf**2
aSf = 1/dyf**2
a0f = -(2/dxf**2 + 2/dyf**2)

phif, Sf = prepare_phi_and_S(Nxf, Nyf, phif, L,H)

for _ in tqdm(range(10000)):
    R2f_old,_,_ = residual(Nxf,Nyf,phif,Sf,aEf,aWf,aNf,aSf,a0f, convert=False)
    R2f = 1e10
    while R2f/R2f_old > 0.5:
        # One GS sweep on fine mesh
        phif = GS(Nxf,Nyf,phif,Sf,aEf,aWf,aNf,aSf,a0f)
        
        R2f,Rsumf,Rf_new = residual(Nxf,Nyf,phif,Sf,aEf,aWf,aNf,aSf,a0f, convert=False)

    # Transfer Residual to corse mesh


    
    if _ %500==0:
        clear_output(True)
        print('Residual:', R2)
        
    if R2<tol:
        print('Converged! Residual: ',R2, 'Time elapsed: ', time.time()-start)
        break
